In [5]:
!pip uninstall pandas y


The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Would remove:
    /homes/hans/.local/lib/python3.10/site-packages/pandas-2.2.3.dist-info/*
    /homes/hans/.local/lib/python3.10/site-packages/pandas/*
Proceed (Y/n)? ERROR: Operation cancelled by user
^C


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

import ast

In [23]:
import pandas as pd
import numpy as np
import ast
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Load CSVs
# clip_df = pd.read_csv("\\medias\db\ImagingSecurity_misc\hans\CLIP_Image_Keywording-main\CLIP_Image_Keywording-main\emb_alex\keyword_1.csv")
selected_csvs = [
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_alec/keywords_extractions_with_embeddings1.csv",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_alec/keywords_extractions_with_embeddings2.csv",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_sahar/keywords_extractions_with_embeddings1.csv",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_sahar/keywords_extractions_with_embeddings2.csv",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_sahar/keywords_extractions_with_embeddings3.csv",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_andy_coco/keywords_extractions_with_embeddings1.csv",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_andy_defacto/keywords_extractions_with_embeddings1.csv",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_abdel/keywords_extractions_with_embeddings_fake.csv"





    
]

# Read and merge them
clip_df = pd.concat([pd.read_csv(f) for f in selected_csvs], ignore_index=True)
label_df = pd.read_csv("/medias/db/ImagingSecurity_misc/Collaborations/ImVerif_Detector 2/output/labels.csv")




In [3]:
clip_df.rename(columns={"images": "image_path", "imagepath": "image_path"}, inplace=True)

# Merge on image_path
data_df = pd.merge(clip_df, label_df, on="image_path")

In [4]:
# Merge on image_path
# data_df = pd.merge(clip_df, label_df, on="image_path")
# data_df = pd.merge(clip_df, label_df, left_on="images", right_on="image_path")

In [5]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387573 entries, 0 to 387572
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   image_path     387573 non-null  object
 1   keyword_1      387573 non-null  object
 2   keyword_2      387573 non-null  object
 3   keyword_3      387573 non-null  object
 4   keyword_4      387573 non-null  object
 5   keyword_5      387573 non-null  object
 6   keyword_6      387573 non-null  object
 7   keyword_7      387573 non-null  object
 8   keyword_8      387573 non-null  object
 9   keyword_9      387573 non-null  object
 10  keyword_10     387573 non-null  object
 11  embedding_1    387573 non-null  object
 12  embedding_2    387573 non-null  object
 13  embedding_3    387573 non-null  object
 14  embedding_4    387573 non-null  object
 15  embedding_5    387573 non-null  object
 16  embedding_6    387573 non-null  object
 17  embedding_7    387573 non-null  object
 18  embe

In [6]:
data_df['correct_label'].value_counts()


correct_label
0    249951
1    137622
Name: count, dtype: int64

In [7]:
# import numpy as np
# import ast
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler


# # Assuming 'data' is your DataFrame
# embedding_columns = [col for col in data_df.columns if col.startswith('embedding_')]

# # Convert the string embeddings into actual arrays
# X = np.array([np.array(ast.literal_eval(embed)) for embed in data_df[embedding_columns].values.flatten()]).astype(np.float32)


# # Assuming all columns except 'image_path' and 'label' are embeddings
# # embedding_cols = [col for col in data_df.columns if col.startswith("embedding_")]
# # X = data_df[embedding_cols].values.astype(np.float32)
# y = data_df['correct_label'].values.astype(float)

# # Standardize embeddings
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# # Train/test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Convert to PyTorch tensors
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
# X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test, dtype=torch.float32)



In [8]:
# X = data_df[embedding_columns].values.astype(np.float32)

embedding_columns = [col for col in data_df.columns if col.startswith("embedding_")]

# Parse and concatenate row-wise embeddings
X = np.vstack([
    np.hstack([ast.literal_eval(str(row[col])) for col in embedding_columns])
    for _, row in data_df.iterrows()
])
y = data_df['correct_label'].values.astype(float)

# Standardize
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [10]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64)


In [11]:
import torch.nn as nn

class MLPClassifier(nn.Module):
    def __init__(self, input_dim):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)  
        )

    def forward(self, x):
        return self.model(x)

model = MLPClassifier(input_dim=X.shape[1])


In [12]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

n_epochs = 30
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb).squeeze()
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # accuracy
        probs = torch.sigmoid(preds)
        predicted = (probs > 0.5).float()
        correct += (predicted == yb).sum().item()
        total += yb.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}, Accuracy = {acc:.4f}")


Epoch 1: Loss = 1800.7602, Accuracy = 0.8025
Epoch 2: Loss = 1677.0553, Accuracy = 0.8161
Epoch 3: Loss = 1610.2826, Accuracy = 0.8237
Epoch 4: Loss = 1548.9440, Accuracy = 0.8294
Epoch 5: Loss = 1489.1277, Accuracy = 0.8363
Epoch 6: Loss = 1439.6335, Accuracy = 0.8415
Epoch 7: Loss = 1386.1417, Accuracy = 0.8468
Epoch 8: Loss = 1330.5148, Accuracy = 0.8535
Epoch 9: Loss = 1287.1857, Accuracy = 0.8595
Epoch 10: Loss = 1241.5456, Accuracy = 0.8636
Epoch 11: Loss = 1198.5066, Accuracy = 0.8696
Epoch 12: Loss = 1156.5519, Accuracy = 0.8740
Epoch 13: Loss = 1118.2856, Accuracy = 0.8795
Epoch 14: Loss = 1084.8622, Accuracy = 0.8838
Epoch 15: Loss = 1051.8201, Accuracy = 0.8885
Epoch 16: Loss = 1017.3311, Accuracy = 0.8926
Epoch 17: Loss = 985.1922, Accuracy = 0.8974
Epoch 18: Loss = 953.1277, Accuracy = 0.9008
Epoch 19: Loss = 931.2102, Accuracy = 0.9037
Epoch 20: Loss = 898.0821, Accuracy = 0.9078
Epoch 21: Loss = 875.4146, Accuracy = 0.9109
Epoch 22: Loss = 852.3821, Accuracy = 0.9130
Epo

In [13]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb).squeeze()
        probs = torch.sigmoid(preds)
        predicted = (probs > 0.5).float()
        correct += (predicted == yb).sum().item()
        total += yb.size(0)

print(f"Test Accuracy: {correct / total:.4f}")


Test Accuracy: 0.8210


In [14]:
model.eval()
results = []

with torch.no_grad():
    for i, (xb, yb) in enumerate(test_loader):
        preds = model(xb).squeeze()
        probs = torch.sigmoid(preds)
        predicted = (probs > 0.5).float()

        batch_indices = range(i * test_loader.batch_size, i * test_loader.batch_size + len(xb))
        for j, idx in enumerate(batch_indices):
            image_path = data_df.iloc[test_dataset.indices[idx]]['image_path']
            score = probs[j].item()
            pred_label = int(predicted[j].item())
            correct_label = int(yb[j].item())
            results.append([image_path, score, pred_label, correct_label])

# Save to CSV
results_df = pd.DataFrame(results, columns=["image_path", "score", "predicted_label", "correct_label"])
results_df.to_csv("mlp_test_predictions.csv", index=False)
print("Saved predictions to mlp_test_predictions.csv")


AttributeError: 'TensorDataset' object has no attribute 'indices'

In [15]:
# Load CSVs
# clip_df = pd.read_csv("\\medias\db\ImagingSecurity_misc\hans\CLIP_Image_Keywording-main\CLIP_Image_Keywording-main\emb_alex\keyword_1.csv")
selected_csvs = [
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_alec/keywords_extractions_with_embeddings4.csv"
]
# Read and merge them
clip_df_test = pd.concat([pd.read_csv(f) for f in selected_csvs], ignore_index=True)
label_df_test = pd.read_csv("/medias/db/ImagingSecurity_misc/Collaborations/ImVerif_Detector 2/output/labels.csv")

In [17]:
clip_df_test.rename(columns={"images": "image_path", "imagepath": "image_path"}, inplace=True)

# Merge on image_path
new_data_df = pd.merge(clip_df_test, label_df_test, on="image_path")

In [ ]:
new_data_df['correct_label'].value_counts()


KeyError: 'correct_label'

In [ ]:

# Load new data
# # Parse and concatenate row-wise embeddings
# X = np.vstack([
#     np.hstack([ast.literal_eval(str(row[col])) for col in embedding_columns])
#     for _, row in data_df.iterrows()
# ])
# y = data_df['correct_label'].values.astype(float)
# Parse embeddings
embedding_columns = [col for col in clip_train_df2.columns if col.startswith('embedding_')]

X_new = np.vstack([
    np.hstack([ast.literal_eval(str(row[col])) for col in embedding_columns])
    for _, row in clip_train_df2.iterrows()
])
y_new = clip_train_df2['correct_label'].values.astype(np.float32)

# Standardize with the SAME scaler used during training
X_new = scaler.transform(X_new)

# Convert to tensor
X_new_tensor = torch.tensor(X_new, dtype=torch.float32)
image_paths_new = clip_train_df2['image_path'].values


In [ ]:
# Convert to tensors
X_new_tensor = torch.tensor(X_new, dtype=torch.float32)
y_new_tensor = torch.tensor(y_new, dtype=torch.float32)

# Dataset and loader
class EvalDataset(Dataset):
    def __init__(self, embeddings, labels, paths):
        self.embeddings = embeddings
        self.labels = labels
        self.paths = paths

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx], self.paths[idx]

eval_dataset = EvalDataset(X_new_tensor, y_new_tensor, image_paths_new)
eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False)


In [ ]:
model.eval()
all_preds = []
all_probs = []
all_labels = []
all_paths = []

with torch.no_grad():
    for xb, yb, paths in eval_loader:
        preds = model(xb).squeeze()
        probs = torch.sigmoid(preds)
        predicted = (probs > 0.5).float()

        all_preds.extend(predicted.tolist())
        all_probs.extend(probs.tolist())
        all_labels.extend(yb.tolist())
        all_paths.extend(paths)

# Compute metrics
acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds)
rec = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")



Accuracy: 0.7467
Precision: 0.7303
Recall: 0.7836
F1 Score: 0.7560


In [ ]:
results_df = pd.DataFrame({
    "image_path": all_paths,
    "score": all_probs,
    "predicted_label": all_preds,
    "correct_label": all_labels
})

results_df.to_csv("new_dataset_evaluation.csv", index=False)
print("Saved evaluation results to new_dataset_evaluation.csv")


Saved evaluation results to new_dataset_evaluation.csv


In [50]:
folders = [
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_alec",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_extranew",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_andy_coco",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_andy_defacto",
    "/medias/db/ImagingSecurity_misc/hans/CLIP_Image_Keywording-main/CLIP_Image_Keywording-main/emb_abdel"


    
]

In [51]:
# Read and merge them
# clip_df = pd.concat([pd.read_csv(f) for f in selected_csvs], ignore_index=True)
# label_df = pd.read_csv("/medias/db/ImagingSecurity_misc/Collaborations/ImVerif_Detector 2/output/labels.csv")

In [52]:
import glob
import os

csv_files = []
for folder in folders:
    csv_files.extend(glob.glob(os.path.join(folder, "*.csv")))

# Load and concatenate all CSVs
dataframes = [pd.read_csv(file) for file in csv_files]
merged_df = pd.concat(dataframes, ignore_index=True)

# Save merged result
# merged_df.to_csv("merged_dataset.csv", index=False)
print(f"Merged {len(csv_files)} files into 'merged_dataset.csv'")

ParserError: Error tokenizing data. C error: out of memory

In [ ]:
merged_df

,image_path,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,keyword_9,...,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10
0,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,loose tooth - dental health,cosmetic dentistry,follicular thyroid cancer,breast cancer research foundation,periodontal disease,tooth whitening,civil partnership,female likeness,human gums,...,"0.16689748,0.24462721,-0.27516177,0.19696245,-...","0.1256808,0.045402173,-0.17783809,0.18022755,-...","0.101030245,0.005906716,-0.27579218,-0.0763301...","0.21998347,-0.47620937,-0.023138404,-0.0753918...","0.11858778,0.30932975,-0.2792756,0.19215856,-0...","0.028180074,-0.011604641,-0.036027476,0.104113...","0.05469504,-0.13024452,-0.07554264,-0.0726553,...","-0.03874011,0.07529713,-0.15263185,0.04254458,...","0.07650763,0.1328947,-0.14425394,0.15712017,-0...","0.13194975,-0.13902599,-0.12958184,0.14509723,..."
1,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,equestrian show jumping,steeplechasing - horse racing,hurdling - horse racing,equestrian cross-country,horse racing,equestrian eventing,equestrian event,horse cutting competition,belgian horse,...,"0.25808978,-0.049095504,-0.15424718,-0.1723377...","0.10086577,-0.13281974,-0.13271217,-0.05083974...","0.14500459,-0.2653166,-0.10391743,0.029785886,...","-0.06922731,0.111724004,0.0048220865,0.0445027...","0.08368831,-0.19616878,-0.052992143,0.07783467...","0.00551736,-0.007815701,0.024640143,-0.0680095...","0.07287204,0.051291063,0.012911126,0.060270563...","0.026474953,-0.14154738,0.07662231,-0.04015472...","0.05566791,-0.1089879,0.12309444,-0.09569396,-...","0.06259683,0.046993025,-0.01736433,0.1208103,0..."
2,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,handkerchief on head,turban,headscarf,sri lankan ethnicity,climate activist,haircap moss,black-faced spoonbill,tyrolean hat,pakistani ethnicity,...,"-0.14597291,0.005604282,-0.2444261,-0.09718268...","0.042650893,0.1828218,-0.13287811,-0.44167978,...","-0.082577795,-0.2438388,-0.35199377,-0.1787149...","0.22473592,0.010070162,0.08562359,0.4047756,-0...","0.032377362,-0.028149113,-0.16028291,-0.133923...","-0.029640919,-0.14904153,-0.08771253,-0.040698...","0.073769465,-0.04739083,-0.02999692,0.15849149...","-0.014878839,0.22839437,-0.08210488,-0.2369712...","0.20817356,-0.109539576,0.102934405,0.08983797...","-0.035265736,0.15377428,-0.18581274,-0.1216650..."
3,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,tibetan ethnicity,navajo ethnicity,nepalese ethnicity,asian and indian ethnicities,undocumented immigrant,central asian ethnicity,sioux ethnicity,southeast asian ethnicity,peruvian ethnicity,...,"0.0868645,0.3040141,0.16031498,-0.008706756,0....","-0.28542954,-0.03331049,-0.05502721,0.072439,-...","0.17622356,-0.18848161,-0.08761207,-0.21820807...","-0.21943708,-0.027840614,-0.08637966,0.2545274...","-0.18608356,0.004852671,-0.14379664,0.21339396...","-0.13636175,0.05759576,-0.12647629,0.21050307,...","-0.25806215,0.06782043,-0.1653953,-0.039335214...","-0.1723036,-0.13164487,-0.18828413,0.19166738,...","-0.10647546,-0.026969448,0.16609439,-0.0549053...","-0.1643224,0.038801588,-0.11472207,0.11067215,..."
4,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,recreational horseback riding,equestrian cross-country,horseback riding,berchtesgaden,equestrian event,equestrian eventing,horse cutting competition,equestrian show jumping,cascade saddle,...,"-0.02579869,0.09566374,0.10479434,-0.07529171,...","-0.06922731,0.111724004,0.0048220865,0.0445027...","-0.07234226,0.06396318,-0.03383184,-0.00493259...","-0.027167205,-0.034204256,0.3390364,-0.0701261...","0.07287204,0.051291063,0.012911126,0.060270563...","0.00551736,-0.007815701,0.024640143,-0.0680095...","0.026474953,-0.14154738,0.07662231,-0.04015472...","0.25808978,-0.049095504,-0.15424718,-0.1723377...","-0.20269331,-0.036743253,0.16608569,-0.0768483...","-0.24610451,-0.22925353,-0.2037214,0.111

In [ ]:
label_df

,image_path,correct_label
0,/medias/db/ImagingSecurity_misc/Collaborations...,0
1,/medias/db/ImagingSecurity_misc/Collaborations...,0
2,/medias/db/ImagingSecurity_misc/Collaborations...,0
3,/medias/db/ImagingSecurity_misc/Collaborations...,0
4,/medias/db/ImagingSecurity_misc/Collaborations...,0
...,...,...
1405795,/medias/db/deepfakes/DF40_dataset/deepfakes_de...,0
1405796,/medias/db/deepfakes/DF40_dataset/deepfakes_de...,0
1405797,/medias/db/deepfakes/DF40_dataset/deepfakes_de...,0
1405798,/medias/db/deepfakes/DF40_dataset/deepfakes_de...,0


In [ ]:
merged_df.rename(columns={"images": "image_path", "imagepath": "image_path"}, inplace=True)


In [ ]:

# Merge on image_path
clip_train_df2 = pd.merge(merged_df, label_df, on="image_path", how="inner")

In [ ]:
clip_train_df2

,image_path,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,keyword_9,...,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10,correct_label
0,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,loose tooth - dental health,cosmetic dentistry,follicular thyroid cancer,breast cancer research foundation,periodontal disease,tooth whitening,civil partnership,female likeness,human gums,...,"0.1256808,0.045402173,-0.17783809,0.18022755,-...","0.101030245,0.005906716,-0.27579218,-0.0763301...","0.21998347,-0.47620937,-0.023138404,-0.0753918...","0.11858778,0.30932975,-0.2792756,0.19215856,-0...","0.028180074,-0.011604641,-0.036027476,0.104113...","0.05469504,-0.13024452,-0.07554264,-0.0726553,...","-0.03874011,0.07529713,-0.15263185,0.04254458,...","0.07650763,0.1328947,-0.14425394,0.15712017,-0...","0.13194975,-0.13902599,-0.12958184,0.14509723,...",1
1,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,equestrian show jumping,steeplechasing - horse racing,hurdling - horse racing,equestrian cross-country,horse racing,equestrian eventing,equestrian event,horse cutting competition,belgian horse,...,"0.10086577,-0.13281974,-0.13271217,-0.05083974...","0.14500459,-0.2653166,-0.10391743,0.029785886,...","-0.06922731,0.111724004,0.0048220865,0.0445027...","0.08368831,-0.19616878,-0.052992143,0.07783467...","0.00551736,-0.007815701,0.024640143,-0.0680095...","0.07287204,0.051291063,0.012911126,0.060270563...","0.026474953,-0.14154738,0.07662231,-0.04015472...","0.05566791,-0.1089879,0.12309444,-0.09569396,-...","0.06259683,0.046993025,-0.01736433,0.1208103,0...",0
2,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,handkerchief on head,turban,headscarf,sri lankan ethnicity,climate activist,haircap moss,black-faced spoonbill,tyrolean hat,pakistani ethnicity,...,"0.042650893,0.1828218,-0.13287811,-0.44167978,...","-0.082577795,-0.2438388,-0.35199377,-0.1787149...","0.22473592,0.010070162,0.08562359,0.4047756,-0...","0.032377362,-0.028149113,-0.16028291,-0.133923...","-0.029640919,-0.14904153,-0.08771253,-0.040698...","0.073769465,-0.04739083,-0.02999692,0.15849149...","-0.014878839,0.22839437,-0.08210488,-0.2369712...","0.20817356,-0.109539576,0.102934405,0.08983797...","-0.035265736,0.15377428,-0.18581274,-0.1216650...",1
3,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,tibetan ethnicity,navajo ethnicity,nepalese ethnicity,asian and indian ethnicities,undocumented immigrant,central asian ethnicity,sioux ethnicity,southeast asian ethnicity,peruvian ethnicity,...,"-0.28542954,-0.03331049,-0.05502721,0.072439,-...","0.17622356,-0.18848161,-0.08761207,-0.21820807...","-0.21943708,-0.027840614,-0.08637966,0.2545274...","-0.18608356,0.004852671,-0.14379664,0.21339396...","-0.13636175,0.05759576,-0.12647629,0.21050307,...","-0.25806215,0.06782043,-0.1653953,-0.039335214...","-0.1723036,-0.13164487,-0.18828413,0.19166738,...","-0.10647546,-0.026969448,0.16609439,-0.0549053...","-0.1643224,0.038801588,-0.11472207,0.11067215,...",1
4,/medias/db/ImagingSecurity_misc/sitcharn/ArtiF...,recreational horseback riding,equestrian cross-country,horseback riding,berchtesgaden,equestrian event,equestrian eventing,horse cutting competition,equestrian show jumping,cascade saddle,...,"-0.06922731,0.111724004,0.0048220865,0.0445027...","-0.07234226,0.06396318,-0.03383184,-0.00493259...","-0.027167205,-0.034204256,0.3390364,-0.0701261...","0.07287204,0.051291063,0.012911126,0.060270563...","0.00551736,-0.007815701,0.024640143,-0.0680095...","0.026474953,-0.14154738,0.07662231,-0.04015472...","0.25808978,-0.049095504,-0.15424718,-0.1723377...","-0.20269331,-0.036743253,0.16608569,-0.0768483...","-0.24610451,-0.22925353,-0.2037214,0.11196736,...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095585,/medias/db/ImagingSecurity_misc/elmennao/datas...,blush - make-up,blusher - make-up,best actress,pearl earring,cream colored lipstick,make-up

In [53]:
clip_train_df2['correct_label'].value_counts()


correct_label
0    1165478
1     930112
Name: count, dtype: int64

In [ ]:
# Save model state_dict (recommended)
torch.save(model.state_dict(), "mlp_classifier.pth")
